# Importing staff and loading accounts

In [ ]:
%matplotlib inline
# standard-ish Qiskit libraries
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer, IBMQ

# about transpiling, compiling, and executing
from qiskit.compiler import transpile, assemble
from qiskit.transpiler import CouplingMap, PassManager, Layout
from qiskit.transpiler.passmanager import DoWhileController, ConditionalController, FlowController
from qiskit.transpiler.passes import *

# Standard Gates
from qiskit.extensions.standard import *

# For graphs
from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Aer noise models
from qiskit.providers.aer import noise

# Loading your IBM Q account(s)
IBMQ.load_accounts()

In [ ]:
%qiskit_backend_overview

In [ ]:
ibmqx2 = IBMQ.get_backend('ibmqx2')
%qiskit_backend_monitor ibmqx2

In [ ]:
properties = ibmqx2.properties()
coupling_map = ibmqx2.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(properties)
basis_gates = noise_model.basis_gates
backend = Aer.get_backend('qasm_simulator')

# The mapping problem

In [ ]:
def check_mapper(circuit, coupling_map):
        pm = PassManager([CheckMap(coupling_map), CheckCXDirection(coupling_map)])
        pm.run(circuit)
        print('right qubits?', pm.property_set['is_swap_mapped'])
        print('right direction?', pm.property_set['is_direction_mapped'])
coupling_map = CouplingMap(ibmqx2.configuration().coupling_map)
print(coupling_map)

In [ ]:
qr = QuantumRegister(5, 'q')
cr = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qr, cr)
circuit.h(qr[0])
circuit.cx(qr[0], qr[4])
circuit.measure(qr[0], cr[0])
circuit.measure(qr[4], cr[1])
check_mapper(circuit, coupling_map)
circuit.draw(output='latex')

In [ ]:
pm = PassManager(BasicSwap(coupling_map))
circuit = pm.run(circuit)
check_mapper(circuit, coupling_map)
circuit.draw()

In [ ]:
pm = PassManager(Decompose(SwapGate))
circuit = pm.run(circuit)
check_mapper(circuit, coupling_map)
circuit.draw()

In [ ]:
pm = PassManager(CXDirection(coupling_map))
circuit = pm.run(circuit)
check_mapper(circuit, coupling_map)
circuit.draw()

In [ ]:
job_sim = execute(circuit, backend,
                  coupling_map=coupling_map,
                  noise_model=noise_model,
                  basis_gates=basis_gates)
counts = job_sim.result().get_counts()
plot_histogram(counts)

# Layout selection

In [ ]:
# Same circuit as before
qr = QuantumRegister(5, 'q')
cr = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qr, cr)
circuit.h(qr[0])
circuit.cx(qr[0], qr[4])
circuit.measure(qr[0], cr[0])
circuit.measure(qr[4], cr[1])

pm = PassManager()
pm.append([CheckMap(coupling_map), CheckCXDirection(coupling_map)])
circuit = pm.run(circuit)
print(coupling_map)
print('right qubits?', pm.property_set['is_swap_mapped'])
print('right direction?', pm.property_set['is_direction_mapped'])

circuit.draw()

In [ ]:
layout = Layout({qr[0]: 0,
                 qr[1]: 4,
                 qr[2]: 2,
                 qr[3]: 3,
                 qr[4]: 1})

pm = PassManager(SetLayout(layout))
pm.append([CheckMap(coupling_map), CheckCXDirection(coupling_map)])
circuit = pm.run(circuit)
print(coupling_map)
print('right qubits?', pm.property_set['is_swap_mapped'])
print('right direction?', pm.property_set['is_direction_mapped'])

circuit = pm.run(circuit)
circuit.draw()

In [ ]:
job_sim = execute(circuit, backend,
                  coupling_map=coupling_map,
                  noise_model=noise_model,
                  basis_gates=basis_gates)
counts = job_sim.result().get_counts()
plot_histogram(counts)

# Control flow in pass managers

## Conditional

In [ ]:
# Same circuit as before
qr = QuantumRegister(5, 'q')
cr = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qr, cr)
circuit.h(qr[0])
circuit.cx(qr[0], qr[4])
circuit.measure(qr[0], cr[0])
circuit.measure(qr[4], cr[1])

layout = Layout({qr[0]: 0,
                 qr[1]: 4,
                 qr[2]: 2,
                 qr[3]: 3,
                 qr[4]: 1})

pm = PassManager(SetLayout(layout))
pm.append([CheckMap(coupling_map), CheckCXDirection(coupling_map)])
circuit = pm.run(circuit)
print(coupling_map)
print('right qubits?', pm.property_set['is_swap_mapped'])
print('right direction?', pm.property_set['is_direction_mapped'])

circuit = pm.run(circuit)
circuit.draw()

In [ ]:
# Try different layouts and observe the results
layout = Layout({qr[0]: 0,
                 qr[1]: 4,
                 qr[2]: 2,
                 qr[3]: 3,
                 qr[4]: 1})

pm = PassManager(SetLayout(layout))
pm.append([CheckMap(coupling_map), CheckCXDirection(coupling_map)])
pm.append(BasicSwap(coupling_map), condition=lambda property_set: not property_set['is_swap_mapped'])
pm.append([Decompose(SwapGate), CXDirection(coupling_map)],
          condition=lambda property_set: not property_set['is_direction_mapped'])

circuit_after = pm.run(circuit)
circuit_after.draw()

## Do-while loop

In [ ]:
qr = QuantumRegister(4, 'q')
circuit = QuantumCircuit(qr)
circuit.h(qr)
circuit.cx(qr[0], qr[3])
circuit.cx(qr[0], qr[2])
circuit.cx(qr[0], qr[1])
circuit.cx(qr[0], qr[1])
circuit.h(qr[1])
circuit.cx(qr[0], qr[2])
circuit.h(qr[2])
circuit.cx(qr[0], qr[3])
circuit.h(qr[0])
circuit.h(qr[3])
circuit.draw()

In [ ]:
pm = PassManager()
pm.append(CXCancellation())
pm.run(circuit).draw()

In [ ]:
pm = PassManager()
pm.append([CXCancellation(), Depth(), FixedPoint('depth')],
         do_while=lambda property_set: not property_set['depth_fixed_point'])
pm.run(circuit).draw()

In [ ]:
# Can you reduce this circuit to the minimum depth?
qr = QuantumRegister(4, 'q')
circuit = QuantumCircuit(qr)
circuit.h(qr)
circuit.cx(qr[0], qr[3])
circuit.cx(qr[0], qr[2])
circuit.h(qr[1:3])
circuit.cx(qr[0], qr[1])
circuit.cx(qr[0], qr[1])
circuit.h(qr[1:3])
circuit.h(qr[1])
circuit.cx(qr[0], qr[2])
circuit.h(qr[2])
circuit.cx(qr[0], qr[3])
circuit.h(qr[0])
circuit.h(qr[3])
circuit.draw()
from qiskit.transpiler import passes
[pass_ for pass_ in dir(passes) if pass_[0].isupper()]

# Optimization levels (preset pass managers)

In [ ]:
import math
cmap = CouplingMap([[0, 1], [0, 5], [1, 0], [1, 2], [1, 6], [2, 1],
                [2, 3], [2, 6], [3, 2], [3, 8], [3, 9], [4, 8], [4, 9],
                [5, 0], [5, 6], [5, 10], [5, 11], [6, 1], [6, 2], [6, 5],
                [6, 7], [6, 10], [6, 11], [7, 1], [7, 6], [7, 8], [7, 12],
                [7, 13], [8, 3], [8, 4], [8, 7], [8, 9], [8, 12], [8, 13],
                [9, 3], [9, 4], [9, 8], [10, 5], [10, 6], [10, 11], [10, 15],
                [11, 5], [11, 6], [11, 10], [11, 12], [11, 16], [11, 17],
                [12, 7], [12, 8], [12, 11], [12, 13], [12, 16], [13, 7],
                [13, 8], [13, 12], [13, 14], [13, 18], [13, 19], [14, 13],
                [14, 18], [14, 19], [15, 10], [15, 16], [16, 11], [16, 12],
                [16, 15], [16, 17], [17, 11], [17, 16], [18, 13], [18, 14],
                [19, 13], [19, 14]]) # Tokyo backend
qr = QuantumRegister(10,'q')
qc = QuantumCircuit(qr)
random_state = [
    1 / math.sqrt(4) * complex(0, 1),
    1 / math.sqrt(8) * complex(1, 0),
    0,
    0,
    0,
    0,
    0,
    0,
    1 / math.sqrt(8) * complex(1, 0),
    1 / math.sqrt(8) * complex(0, 1),
    0,
    0,
    0,
    0,
    1 / math.sqrt(4) * complex(1, 0),
    1 / math.sqrt(8) * complex(1, 0)]
qc.initialize(random_state, qr[0:4])
pm = PassManager(Unroller(basis_gates))
pm.append([CheckMap(cmap), CheckCXDirection(cmap)])
qc_original = pm.run(qc)
print('right qubits?', pm.property_set['is_swap_mapped'])
print('right direction?', pm.property_set['is_direction_mapped'])
print('gates = ', qc_original.count_ops())
print('depth = ', qc_original.depth())
qc_original.draw()

In [ ]:
optimized_0 = transpile(qc,
                        coupling_map=cmap,
                        basis_gates=basis_gates,
                        seed_transpiler=11,
                        optimization_level=0)
# check_mapper(optimized_0, cmap)
print('gates = ', optimized_0.count_ops())
print('depth = ', optimized_0.depth())
optimized_0.draw()

In [ ]:
optimized_1 = transpile(qc,
                        coupling_map=cmap,
                        basis_gates=basis_gates,
                        seed_transpiler=11,
                        optimization_level=1)
check_mapper(optimized_1, cmap)
print('gates = ', optimized_1.count_ops())
print('depth = ', optimized_1.depth())
optimized_1.draw()

In [ ]:
optimized_2 = transpile(qc,
                        coupling_map=cmap,
                        basis_gates=basis_gates,
                        seed_transpiler=42,
                        optimization_level=2)
check_mapper(optimized_2, cmap)
print('gates = ', optimized_2.count_ops())
print('depth = ', optimized_2.depth())
optimized_2.draw()

In [ ]:
optimized_3 = transpile(qc,
                        coupling_map=cmap,
                        basis_gates=basis_gates,
                        seed_transpiler=42,
                        optimization_level=3)
check_mapper(optimized_3, cmap)
print('gates = ', optimized_3.count_ops())
print('depth = ', optimized_3.depth())
optimized_3.draw()